In [1]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)

In [2]:
print(faces.target_names)
print(faces.images.shape)

['Ariel Sharon' 'Colin Powell' 'Donald Rumsfeld' 'George W Bush'
 'Gerhard Schroeder' 'Hugo Chavez' 'Junichiro Koizumi' 'Tony Blair']
(1348, 62, 47)


In [4]:
%matplotlib inline 
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np 

C:\Users\HP\AppData\Local\Temp\ipykernel_3840\3597052998.py:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [5]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=150,whiten=True,svd_solver='randomized',random_state=42)
svc = SVC(kernel='rbf',class_weight='balanced')
model = make_pipeline(pca, svc)

In [6]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest = train_test_split(faces.data,faces.target,random_state=42)

In [7]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C':[1,5,10,50],
              'svc__gamma':[0.0001,0.0005,0.001,0.005]}
grid = GridSearchCV(model,param_grid)

%time grid.fit(Xtrain,ytrain)

CPU times: total: 1min 36s
Wall time: 1min 10s


GridSearchCV(estimator=Pipeline(steps=[('pca',
                                        PCA(n_components=150, random_state=42,
                                            svd_solver='randomized',
                                            whiten=True)),
                                       ('svc', SVC(class_weight='balanced'))]),
             param_grid={'svc__C': [1, 5, 10, 50],
                         'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]})

In [10]:
model = grid.best_estimator_

In [35]:
Xtest[[0]]

array([[0.47189546, 0.524183  , 0.6039216 , ..., 0.41830066, 0.78823525,
        0.94640523]], dtype=float32)

In [36]:
model.predict(Xtest[[4]])

array([6], dtype=int64)

In [38]:
import pickle

with open("model01.pkl", "wb") as file:
    pickle.dump(model, file)

In [39]:
Xtest[[0]].shape

(1, 2914)

In [46]:
import cv2
img = cv2.imread("D:/Lec2023/Campus/4thYear/EES/ML/image/person_2.jpg",cv2.IMREAD_GRAYSCALE)
img_1d = img.flatten()
img_2d = np.array([img_1d],dtype=np.float32)
min_value = img_2d.min()
max_value = img_2d.max()

# Rescale the values to the range [0, 1]
img_rescaled = (img_2d - min_value) / (max_value - min_value)
print(img_rescaled)
pred = model.predict(img_rescaled)
print(pred)

[[0.2901961  0.35686275 0.24705882 ... 0.40784314 0.75686276 1.        ]]
[3]
